In [4]:

import tqdm

import numpy as np
import pandas as pd
import geopandas as gpd

import networkx as nx


In [5]:
# Bus location
bus_loc = pd.read_csv('original_data/bus_stop_2024-09-19.csv',
        dtype={'BusStopCode': str}) \
    .rename(columns={
        'BusStopCode'  : 'bus_stop_code',
        'RoadName'     : 'road_name',
        'Description'  : 'stop_name',
        'Latitude'     : 'lat',
        'Longitude'    : 'lon'})

# to GeoDataFrame
bus_loc = gpd.GeoDataFrame(
        bus_loc, 
        geometry = gpd.points_from_xy(x=bus_loc['lon'], y=bus_loc['lat']),
        crs = 'EPSG:4326') \
    .to_crs('EPSG:3414') \
    .assign(
        x = lambda g: g['geometry'].x, 
        y = lambda g: g['geometry'].y)
print('\nShape of bus location:', bus_loc.shape,
      '\nNumber of bus stop:', bus_loc['bus_stop_code'].nunique(),)


# Bus route
bus_route = pd.read_csv('original_data/bus_route_2024-09-19.csv', 
        dtype={'BusStopCode': str, 'ServiceNo': str, 'StopSequence' : int}) \
    .rename(columns={
        'Operator'     : 'bus_operator',
        'BusStopCode'  : 'bus_stop_code',
        'ServiceNo'    : 'bus_service_no',
        'StopSequence' : 'order_no',
        'Direction'    : 'route_direction',
        'Distance'     : 'stop_spacing' })
print('\nShape of bus route:', bus_route.shape)


# bus_service = pd.read_csv('original_data/bus_service_2024-09-19.csv',
#                           dtype={'BusStopCode': str, 'OriginCode': str, 'DestinationCode' : str})


Shape of bus location: (5136, 8) 
Number of bus stop: 5136

Shape of bus route: (25450, 12)


In [6]:
from Singapore.bus_network._historical_project.codes.utils import create_graph_from_dataframe

data = bus_route.merge(bus_loc, on='bus_stop_code')

node_id_col = 'bus_stop_code'
order_col = 'order_no'
mline_id_col = ['bus_operator', 'bus_service_no', 'route_direction']
node_attr_list = ['bus_stop_code', 'x', 'y', 'lat', 'lon', 'road_name', 'stop_name']
edge_attr_list = ['bus_service_no', 'bus_operator', 'route_direction', 'stop_spacing']


# add nodes
# graph = graph_add_nodes_from_dataframe(graph, bus_loc, node_id_col, node_attr_list)
graph = create_graph_from_dataframe(
    data = data,
    node_id_col = node_id_col,
    order_col = order_col,
    mline_id_col = mline_id_col,
    node_attr_list = node_attr_list,
    edge_attr_list = edge_attr_list,
    space='l')

# processing edge attributes
# merge edge attributes, list -> string
for edge_attr in tqdm.tqdm(['bus_service_no', 'bus_operator', 'route_direction'], desc='Merge edge attributes...'):
    for edge in graph.edges():
        _edge_attr_val = list(set(graph.edges[edge][edge_attr]))
        _edge_attr_val.sort()
        _edge_attr_val = list(map(str, _edge_attr_val))
        graph.edges[edge][edge_attr] = ','.join(_edge_attr_val)

# merge edge attributes, 'stop_spacing'
for edge in graph.edges():
    graph.edges[edge]['stop_spacing'] = np.mean(graph.edges[edge]['stop_spacing'])


# save bus network graph
nx.write_gexf(graph, 'bus_network_2024-09-19.gexf')
nx.write_gml(graph, 'bus_network_2024-09-19.gml')

Merge edge attributes...: 100%|██████████| 3/3 [00:00<00:00, 40.99it/s]
